Erzeuge aus den vorhanden Datenreihen 5 zusätzlich Datenreihen

In [18]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import pandas as pd

# Lade die .env Datei
load_dotenv()

# Setze den OpenAI API Key als Umgebungsvariable
openai_api_key = os.getenv("OPENAI_API_KEY")

if openai_api_key:
    print("OpenAI API Key erfolgreich gesetzt.")
else:
    print("OpenAI API Key nicht in .env Datei gefunden.")
    exit(1)

# Initialisiere den OpenAI Client
client = OpenAI(api_key=openai_api_key)

# Lade die CSV-Datei
file_path = 'csv_data/uberleitung_daten_16.csv'
df = pd.read_csv(file_path)

# Zeige die ersten Zeilen des Datensatzes an
print(df.head())

# Funktion zum Generieren neuer Datensätze mit LLM und Prompt Engineering
def generate_text_data_with_prompt_engineering(muster_df, n):
    new_data = []
    prompt = (
         "Erstelle aus den vorhandenen fiktive Daten (row) für alle Variablen neue Daten und befolge dabei genau die Anweisungen, die ich dir zusätzlich für bestimmte Variablen gebe: "
        "Regeln: "
        "1. Der <vorname> muss zum <geschlecht> passen. Achte darauf, dass die Namen und Vornamen heterogen sind."
        "2. Fülle <pflegekasse> mit zufälligen Namen von deutschen Krankenkassen. "
        "3. Fülle <versichertennummer> mit einer zufälligen 10-stelligen Zahl, die mit einem Großbuchstaben beginnt. "
        "4. Fülle <entlassung> immer mit 10.09.2024 und <datum_aktuell> immer mit 09.09.2024. "
        "   Wenn <entlassung_wohin> 'nach Hause' ist, muss <leistung> immer entweder 'Sachleistung' oder 'Kombinationsleistung' sein. Keine anderen Optionen sind erlaubt. "
        "   Wenn <entlassung_wohin> 'Kurzzeitpflege' oder 'Pflegeeinrichtung' ist, muss <leistung> immer 'Vollstationaer' sein. Keine anderen Optionen sind erlaubt. "
        "   Wenn <entlassung_wohin> 'Rehaklinik' ist, muss <reha> immer mit 'ja' gefüllt werden. "
        "6. Fülle <krankenhaus> mit einem Krankenhaus in der Nähe des <wohnort>. "
        "7. <betreuer> und <hausarzt> können leer bleiben. "
        "8. Fülle <reha> abhängig von <pflege_diagnose>: Wenn die Diagnose eine Reha erforderlich macht, kann <reha> zufällig 'ja' sein, sonst immer 'nein'. "
        "9. Fülle <signature> immer mit dem Inhalt der Variable <name>. "
    )
    
    # Füge mehr Beispiele aus dem DataFrame hinzu (z.B. die ersten 10 Datensätze)
    for index, row in muster_df.head(10).iterrows():
        prompt += ", ".join([f"{col}: {row[col]}" for col in muster_df.columns]) + "\n"
    
    prompt += (
        "\nErzeuge neue Datensätze mit fiktiven Daten, die dem gleichen Muster folgen und beachte dabei genau meine Anweisungen:\n\n"
    )
    
    for _ in range(n):
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Du bist Experte für Data Science."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=500,
            temperature=0.2,
        )
        
        generated_text = response.choices[0].message.content.strip()
        rows = generated_text.split("\n")
        new_row = {col: None for col in muster_df.columns}
        
        for row in rows:
            entries = row.split(", ")
            for entry in entries:
                try:
                    key, value = entry.split(": ")
                    if key in new_row:
                        new_row[key] = value
                except ValueError:
                    print(f"Fehlerhafte Eingabe: {entry}")
                    continue
        
        # Überprüfen, ob der neue Datensatz alle Spalten enthält
        if None not in new_row.values():
            new_data.append(new_row)
        else:
            print(f"Inkompletter Datensatz: {new_row}")
            # Ergänzen fehlender Daten mit weiteren GPT-3.5 Aufrufen
            for col in new_row:
                if new_row[col] is None:
                    completion = client.chat.completions.create(
                        model="gpt-3.5-turbo",
                        messages=[
                            {"role": "system", "content": "Du bist Experte für Data Science."},
                            {"role": "user", "content": f"Ergänze die fehlenden Information mit fiktiven Daten: {col} für den Datensatz: {new_row}. Achte dass jede Reihe sinnvoll ergänzt wurde."}
                        ],
                        max_tokens=80,
                        temperature=0.2,
                    )
                    new_row[col] = completion.choices[0].message.content.strip()
            new_data.append(new_row)
        
    return pd.DataFrame(new_data)

# Generiere 5 neue Datensätze
random_data = generate_text_data_with_prompt_engineering(df, 5)

# Füge die neuen Datensätze zu den ursprünglichen hinzu
complete_data = pd.concat([df, random_data], ignore_index=True)

# Zeige die ersten 5 Zeilen der neuen Datensätze
print(complete_data.head())

# Speichere die neuen Datensätze in einer CSV-Datei
complete_data.to_csv('erweiterte_daten_10.csv', index=False)


OpenAI API Key erfolgreich gesetzt.
          name  vorname   geb_datum geschlecht    wohnort pflegekasse  \
0       Müller    Berta  16.11.1938       frau      Fürth         AOK   
1  Schwarzmann  Christa  23.04.1967       frau   Nürnberg         DAK   
2        Huber     Karl  05.01.1953       mann    München         BKK   
3    Lagerfeld   Judith  12.03.1986       frau   Landshut         AOK   
4     Scharrer  Herbert  25.12.1929       mann  Straubing         AOK   

  versichertennummer  entlassung   entlassung_wohin          krankenhaus  ...  \
0       O89339883998  03.09.2024         nach Hause       Klinikum Fürth  ...   
1           L7878788  04.09.2024         nach Hause  Klinik Martha-Maria  ...   
2         J738383494  05.09.2024  Pflegeeinrichtung  Klinikum Großhadern  ...   
3          J37347347  06.09.2024         nach Hause    Klinikum Landshut  ...   
4        K9394384756  07.09.2024  Pflegeeinrichtung   Klinikum Straubing  ...   

              information             